In [20]:
# %%capture cap --no-stderr

# with open('train.txt', 'w') as f:
#     f.write(cap.stdout)

# del cap 

from itertools import product
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

# components = [ 25, 50, 100 ]
components = [ 50 ]
y = pickle.load(open('target.pkl', 'r'))

best_accuracy_overall = 0
best_params_overall = None
y_pred_best = None

for c in components:
    
    print '-------------------------------------------------------'
    print
    print 'Training and optimisation report with {0} principal component'.format(c)
    print
    filename = 'data-{0}-components.pkl'.format(c)
    X = pickle.load(open(filename, 'r'))

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.14, random_state=0)

    y_pred = np.empty(len(y_test), dtype=np.int)
    
    #n_neighbors_param = [2, 3, 4, 5]
    n_neighbors_param = [5]
    
    for n in n_neighbors_param:
        
        print 'KNN with {0} neighbours'.format(n)
        
        knn_clf = KNeighborsClassifier(n_neighbors=int('{}'.format(n)))
        knn_clf.fit(X_train, y_train)
        
        kneighbors = knn_clf.kneighbors(X_test, return_distance=False)
        
        best_accuracy = 0
        best_params = []
        
        #c_param = [1,10,100,1000]
        c_param = [1]        
        #degree_param = [3,4,5,6]
        degree_param = [3]
        #gamma_param = [1e-3, 1e-4]
        gamma_param = [1e-3]
        #kernel_param = ['poly', 'rbf']
        kernel_param = ['poly']
        for params in product(c_param, degree_param, gamma_param, kernel_param):
            
            c = params[0]
            degree = params[1]
            gamma = params[2]
            kernel = params[3]
        
            for idx, indices in enumerate(kneighbors):
                neighbors = [X_train[i] for i in indices]
                neighbors_labels = [y_train[i] for i in indices]

                if len(set(neighbors_labels)) == 1:
                    y_pred[idx] = neighbors_labels[0]
                else:
                    svm_clf = svm.SVC(kernel='{}'.format(kernel), degree=int('{}'.format(degree)), gamma=float('{}'.format(gamma)), C=int('{}'.format(c)), random_state=0)
                    svm_clf.fit(neighbors, neighbors_labels)
                    label = svm_clf.predict(X_test[idx].reshape(1, -1))
                    y_pred[idx] = label
            
#             print accuracy_score(y_train, y_pred)
            accuracy = accuracy_score(y_test, y_pred)
#             print accuracy, (c, degree, gamma, kernel)
            if accuracy >= best_accuracy:
                best_accuracy = accuracy
                best_params.append((c, degree, gamma, kernel))
                
                if best_accuracy >= best_accuracy_overall:
                    
                    clf_confusion_matrix = confusion_matrix(y_test, y_pred)
                    
                    print clf_confusion_matrix
                    
                    def plot_confusion_matrix(cm, classes, title, cmap=plt.cm.Reds):
                        """
                        This function prints and plots the confusion matrix.
                        Normalization can be applied by setting `normalize=True`.
                        """
                        plt.imshow(cm, interpolation='nearest', cmap=cmap)
                        plt.title(title)
                        plt.colorbar()
                        tick_marks = np.arange(len(classes))
                        plt.xticks(tick_marks, classes)
                        plt.yticks(tick_marks, classes)

                        thresh = cm.max() / 2.
                        for i, j in product(range(cm.shape[0]), range(cm.shape[1])):
                            plt.text(j, i, cm[i, j],
                                     horizontalalignment="center",
                                     color="white" if cm[i, j] > thresh else "black")

                        plt.tight_layout()
                        plt.ylabel('True label')
                        plt.xlabel('Predicted label')

                    # Compute confusion matrix
                    cnf_matrix = confusion_matrix(y_test, y_pred)
                    np.set_printoptions(precision=2)

                    # Plot non-normalized confusion matrix
                    plt.figure()
                    plot_confusion_matrix(cnf_matrix, classes=[0,1,2,3,4,5,6,7,8,9], title='Predicted vs True label')

                    plt.show()
                    
                    
                    error_per_digit = []
                    label = 0 
                    for row in clf_confusion_matrix:
                        num = float(row[label])
                        den = sum(row)
                        error = 1 - (num / den)
                        error_per_digit.append(error)
                        label += 1

                    for e in error_per_digit:
                        print e
                    
                    best_accuracy_overall = best_accuracy
                    best_params_overall = (n, c, degree, gamma, kernel)
                    y_pred_best = y_pred
        
        print 'accuracy:', best_accuracy
        print 'paramters:', best_params
        print
    
    print '-------------------------------------------------------'

print 'best_accuracy_overall:', best_accuracy_overall
print 'paramters:', best_params_overall
print accuracy_score(y_test, y_pred_best)

params_best_file = 'params_best.pkl'
file = open(params_best_file, 'w')
pickle.dump(params_best, file)
file.close()
    
y_pred_best_file = 'y_pred_best.pkl'
file = open(y_pred_best_file, 'w')
pickle.dump(y_pred_best, file)
file.close()

y_test_file = 'y_test_best.pkl'
file = open(y_test_file, 'w')
pickle.dump(y_test, file)
file.close()
     
 

-------------------------------------------------------

Training and optimisation report with 50 principal component

KNN with 5 neighbours


ValueError: Found input variables with inconsistent numbers of samples: [60199, 9801]